<a href="https://colab.research.google.com/github/haru1489248/nlp-100-nock/blob/main/ch09/section_84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
!pip install -U transformers torchmetrics

In [61]:
import torch
from torchmetrics.functional import pairwise_cosine_similarity
from transformers import AutoModel, AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

# Dropoutなどの学習時専用挙動を無効化する
model.eval()

In [63]:
sentences = [
    "The movie was full of fun.",
    "The movie was full of excitement.",
    "The movie was full of crap.",
    "The movie was full of rubbish."
]

### tokenizerについて

tokenizer は複数文（バッチ入力）に対応している。

- sentences: 入力文の配列（`list[str]`）
- return_tensors: 出力を PyTorch などのテンソル形式に変換する指定
- padding: バッチ内で文の長さを `[PAD]` トークンで揃えるかどうか
- truncation: モデルの最大長を超える文章を自動で切り落とすかどうか


In [64]:
inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True)

In [65]:
with torch.no_grad():
  outputs = model(**inputs) # **はキーワード引数として展開するpythonの記法

### last_hiddenはどうなっているか
`last_hidden.shape = (batch_size, seq_len, hidden_size)`
- `batch_size`: 文の数（sentencesの数）
- `seq_len`: トークンの数（[CLS], 単語, [SEP], PAD 含む）
- `hidden_size`: 書くトークンの埋め込み次元（bert-baseなら768）


In [66]:
last_hidden = outputs.last_hidden_state # (B, S, H)
attention_mask = inputs['attention_mask'] # (B, S)
input_ids = inputs['input_ids'] # (B, S)

attention_maskは今回は全て同じ長さの文なのですべて1になる

In [67]:
print(last_hidden)
print(attention_mask)
print(input_ids)

tensor([[[ 0.2828, -0.2636,  0.1093,  ..., -0.4473,  0.3723,  0.2192],
         [ 0.1627, -0.7891, -0.0345,  ..., -0.5373,  0.7820, -0.3398],
         [ 0.9971, -0.4762,  0.3013,  ..., -0.3349,  0.1641, -0.3276],
         ...,
         [ 0.3238, -0.2024,  0.3026,  ..., -0.3850,  0.3314, -0.2486],
         [ 0.8131, -0.0029, -0.2271,  ...,  0.1727, -0.5665, -0.3684],
         [ 0.6521,  0.0869,  0.2177,  ...,  0.2843, -0.5923, -0.1937]],

        [[ 0.1435, -0.2701,  0.1080,  ..., -0.4082,  0.4033,  0.1364],
         [ 0.2259, -0.7829, -0.0506,  ..., -0.4750,  0.7534, -0.2578],
         [ 1.1483, -0.4832,  0.3087,  ..., -0.2553,  0.1238, -0.3381],
         ...,
         [ 0.4517, -0.6837,  0.2091,  ..., -0.8668,  0.5381, -0.3754],
         [ 0.6197,  0.0082, -0.1377,  ...,  0.1973, -0.3866, -0.3518],
         [ 0.3631,  0.1735,  0.4880,  ...,  0.3721, -0.4032, -0.0448]],

        [[ 0.3010,  0.1436, -0.0652,  ..., -0.2415,  0.4671,  0.1211],
         [ 0.6523, -0.5456, -0.1248,  ..., -0

clsのinput_idとsepのinput_idとpadのinput_idを取得する

In [68]:
cls_id = tokenizer.cls_token_id
sep_id = tokenizer.sep_token_id
pad_id = tokenizer.pad_token_id

In [69]:
print(cls_id)
print(sep_id)
print(pad_id)

101
102
0


トークンのinput_idだけを取得するためのmask行列を作成する
最終層の隠れ状態行列は(B, S, H)なので(B, S)から(B, S, 1)に変換する
- unsqueezeは指定した位置にサイズ1の次元を追加する
- boolean値を持つ行列になる

In [70]:
token_mask = (
    (attention_mask == 1) &
    (input_ids != cls_id) &
    (input_ids != sep_id) &
    (input_ids != pad_id)
).unsqueeze(-1)

### dim=1のsumとは？
- sumとはその名の通り合計する関数
- dim=1を指定すると、0始まりで数えた1番目の軸、
  すなわちトークン列（seq_len）方向に対して合計する。


In [71]:
sum_vecs = (last_hidden * token_mask).sum(dim=1)

In [72]:
count = token_mask.sum(dim=1).clamp(min=1) # 0除算を避ける為、1より小さい場合は1に設定する

In [73]:
mean_vecs = sum_vecs / count

デフォルトで自分自身とのコサイン類似度は0になる様に設定されている
- zero_diagonal: Falseで対角線の値（自分自身とのコサイン類似度を1にする様に設定できる）デフォルトはTrue

In [74]:
print(pairwise_cosine_similarity(mean_vecs, zero_diagonal=False))

tensor([[1.0000, 0.9535, 0.8379, 0.8046],
        [0.9535, 1.0000, 0.8256, 0.7849],
        [0.8379, 0.8256, 1.0000, 0.9182],
        [0.8046, 0.7849, 0.9182, 1.0000]])
